# Emblaze

Emblaze is a visualisation tool for embedding spaces. 

When using emblaze the following is needed:

- X, which is our feature data (the model)
- Y, which is label data

emb = emblaze.Embedding({emblaze.Field.POSITION: X, emblaze.Field.COLOR: Y}, metric='cosine')



In [1]:
import csv
import emblaze
from gensim.models import Word2Vec
import numpy as np
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
import re
import emblaze
from emblaze.utils import Field, ProjectionTechnique

In [2]:
# model_5000 = Word2Vec.load("../Data/Models/word2vec_epoch_5000.model")
#model_200 = Word2Vec.load("../Data/Models/word2vec_epoch_200_min_1.model")
# model_1000 = Word2Vec.load("../Data/Models/word2vec_epoch_1000_min_1.model")
#model_3000 = Word2Vec.load("../Data/Models/word2vec_epoch_3000_min_1.model")
model = Word2Vec.load("../Data/Models/best_model.model")


In [3]:

with open('../Data/sequences.csv', 'r') as f:
    reader = csv.reader(f)
    sequences = list(reader)
    
address_count_seq = {}
for seq in sequences:
    for place in seq:
        if place in address_count_seq:
            address_count_seq[place] += 1
        else:
            address_count_seq[place] = 1

address_count_seq

def label_function_address_count(labels, station_names): 
    for station_name in station_names:
            # Check if the station_name exists in address_coordinates dictionary
            if station_name in address_count_seq:
                labels.append(address_count_seq[station_name])
                #if address_count_seq[station_name] == 1:
                    #labels.append(address_count_seq[station_name])
                #else: 
                 #labels.append("")
            #     labels.append(address_count_seq[station_name])
            else: 
                 labels.append("")
    return labels

labels = label_function_address_count([], model.wv.index_to_key)
# Convert labels to numerical format
label_encoder = LabelEncoder()
label_encoder.fit(labels)
numeric_labels = label_encoder.transform(labels)

# X is an n x k array, Y is a length-n array
X, Y = model.wv.vectors, labels

# Represent the high-dimensional embedding
emb = emblaze.Embedding({Field.POSITION: X, Field.COLOR: Y})
# Compute nearest neighbors in the high-D space (for display)
emb.compute_neighbors(metric='cosine')

# Generate UMAP 2D representations - you can pass UMAP parameters to project()
variants = emblaze.EmbeddingSet([
    emb.project(method=ProjectionTechnique.UMAP) for _ in range(10)
])
# Compute neighbors again (to indicate that we want to compare projections)
variants.compute_neighbors(metric='euclidean')

w = emblaze.Viewer(embeddings=variants)
w

FileNotFoundError: [Errno 2] No such file or directory: '../Data/sequences.csv'

In [3]:
f = open('../Data/data.json')
data = json.load(f)

a = open('../Data/added_data.json')
added_data = json.load(a)

file = open("../sequences.csv", "r")
sequences = list(csv.reader(file, delimiter=","))
file.close()


In [6]:
#retrieve zipcode
def getZip(info):
    pattern_zip = '\d{4}'
    pattern_denmark = 'anmark'
    match_zip = re.search(pattern_zip, info)
    match_denmark = re.search(pattern_denmark, info)
    if match_zip is not None and match_denmark is not None:
        return match_zip.group()
    else :
        return

# # Extract zip codes and corresponding coordinates
#zip_coordinates = defaultdict(list)
address_coordinates = {}
wrong_addresses = []
# we have some json objects where the address exists but the info is null. This happens in 1602 cases (assume this is where the geolocator failed)
for address, info in data.items():
    if info is not None: 
        zip_code = getZip(info['address'])  # Extract zip code from address could be empty
        if zip_code is not None:
            if 1000 <= int(zip_code): #for now removing all failed zipcodes in Jylland
                if int(zip_code) <= 1549:
                    #Kbh K
                    address_coordinates[address] = 'Kbh K'
                elif int(zip_code) <= 1799:
                    #Kbh V
                    address_coordinates[address] = 'Vesterbro'
                elif int(zip_code) <= 2000:
                    #Frederiksberg C
                    address_coordinates[address] = 'Frederiksberg'
                elif int(zip_code) == 2400:
                    address_coordinates[address] = 'Kbh NV'
                elif int(zip_code) == 2300:
                    address_coordinates[address] = 'Kbh S'
                elif int(zip_code) == 2200:
                    address_coordinates[address] = 'Nørreport'
                elif int(zip_code) == 2100:
                    address_coordinates[address] = 'Østerbro'
                else:
                    address_coordinates[address] = 'Other'
            else: 
                wrong_addresses.append(address) #go through later
        else:
            address_coordinates[address] = 'Other' #no zipcode was found
    else:
        address_coordinates[address] = 'Other' #no info was found on the address?

#run through the list of wrong addresses and try to find them in the added_data.json file to get the "correct" zipcode
for address in wrong_addresses:
    #find in json file
    if address in added_data:
        zip_code = getZip(added_data[address]['address'])
        if zip_code is None: continue
        if int(zip_code) < 1000:
            continue
        if 1000 <= int(zip_code) < 4999: #for now removing all failed zipcodes in Jylland
            if int(zip_code) <= 1549:
                #Kbh K
                address_coordinates[address] = 'Kbh K'
            elif int(zip_code) <= 1799:
                #Kbh V
                address_coordinates[address] = 'Vesterbro'
            elif int(zip_code) <= 2000:
                #Frederiksberg C
                address_coordinates[address] = 'Frederiksberg'
            elif int(zip_code) == 2400:
                address_coordinates[address] = 'Kbh NV'
            elif int(zip_code) == 2300:
                address_coordinates[address] = 'Kbh S'
            elif int(zip_code) == 2200:
                address_coordinates[address] = 'Nørreport'
            elif int(zip_code) == 2100:
                address_coordinates[address] = 'Østerbro'
            else:
                address_coordinates[address] = 'Other'

def label_function_zipcodes(labels, station_names): 
    for station_name in station_names:
    # Check if the station_name exists in address_coordinates dictionary
        if station_name in address_coordinates:
            zip = address_coordinates[station_name]
            if zip is not None:
                labels.append(zip)
            else: 
                labels.append("unknown")
        else:
            labels.append("unknown")
    return labels

labels = label_function_zipcodes([], model.wv.index_to_key)
# Convert labels to numerical format
label_encoder = LabelEncoder()
label_encoder.fit(labels)
numeric_labels = label_encoder.transform(labels)

In [9]:


# X is an n x k array, Y is a length-n array
X, Y = model.wv.vectors, labels

# Represent the high-dimensional embedding
emb = emblaze.Embedding({Field.POSITION: X, Field.COLOR: Y})
# Compute nearest neighbors in the high-D space (for display)
emb.compute_neighbors(metric='cosine')

# Generate UMAP 2D representations - you can pass UMAP parameters to project()
variants = emblaze.EmbeddingSet([
    emb.project(method=ProjectionTechnique.TSNE) for _ in range(10)
])
# Compute neighbors again (to indicate that we want to compare projections)
variants.compute_neighbors(metric='euclidean')

thumbnails = emblaze.TextThumbnails(model.wv.index_to_key)

w = emblaze.Viewer(embeddings=variants, thumbnails=thumbnails)
w


Viewer(colorScheme='tableau', data={'data': [{'_format': 'compressed', '_idtype': 'u2', '_length': 14820, 'ids…

In [8]:
print('next!')

next!
